<a href="https://colab.research.google.com/github/sooddhruv/PA-HW3/blob/main/PA_HW3_Dhruv_Sood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Breast_Cancer_dataset.csv')


In [ ]:

df.head()

# Check for data types and null values
df.info()
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     3823 non-null   float64
 1   Race                    3622 non-null   object 
 2   Marital Status          3703 non-null   object 
 3   T Stage                 4024 non-null   object 
 4   N Stage                 4024 non-null   object 
 5   6th Stage               4024 non-null   object 
 6   differentiate           4024 non-null   object 
 7   Grade                   4024 non-null   object 
 8   A Stage                 4024 non-null   object 
 9   Tumor Size              3622 non-null   float64
 10  Estrogen Status         3823 non-null   object 
 11  Progesterone Status     4024 non-null   object 
 12  Regional Node Examined  3421 non-null   float64
 13  Reginol Node Positive   4024 non-null   int64  
 14  Survival Months         4024 non-null   

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
count,3823.000000,3622,3703,4024,4024,4024,4024,4024,4024,3622.000000,3823,4024,3421.000000,4024.000000,4024.000000,4024
unique,NaN,3,5,4,3,5,4,4,2,NaN,2,2,NaN,NaN,NaN,2
top,NaN,White,Married,T2,N1,IIA,Moderately differentiated,2,Regional,NaN,Positive,Positive,NaN,NaN,NaN,Alive
freq,NaN,3069,2436,1786,2732,1305,2351,2351,3932,NaN,3567,3326,NaN,NaN,NaN,3408
mean,53.931467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.560188,NaN,NaN,14.351652,4.158052,71.297962,NaN
std,8.972253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.428535,NaN,NaN,8.170800,5.109331,22.921430,NaN
min,30.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,1.000000,1.000000,NaN
25%,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.000000,NaN,NaN,9.000000,1.000000,56.000000,NaN
50%,54.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000,NaN,NaN,14.000000,2.000000,73.000000,NaN
75%,61.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.000000,NaN,NaN,19.000000,5.000000,90.000000,NaN


In [ ]:
# Fill missing values for numerical columns with the median
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

# Fill missing values for categorical columns with the mode
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])


In [ ]:
# Check for any remaining missing values
missing_values = df.isnull().sum()

# Display columns with missing values, if any
missing_values[missing_values > 0]

# Check if there are any missing values across the entire dataset
print("Total missing values:", df.isnull().sum().sum())


Total missing values: 0


In [ ]:
# Dictionary to store outliers for each column
outliers_dict = {}

for col in numerical_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_dict[col] = outliers

    # Print summary of outliers for the column
    print(f"Column: {col}")
    print(f"Number of outliers: {outliers.shape[0]}")
    print(outliers[[col]].head())  # Display first few outliers for inspection
    print("-" * 50)


Column: Age
Number of outliers: 0
Empty DataFrame
Columns: [Age]
Index: []
--------------------------------------------------
Column: Tumor Size
Number of outliers: 138
     Tumor Size
11         59.0
51         55.0
75         55.0
90         60.0
165        62.0
--------------------------------------------------
Column: Regional Node Examined
Number of outliers: 100
     Regional Node Examined
109                    30.0
135                    28.0
225                    27.0
240                    29.0
241                    30.0
--------------------------------------------------
Column: Reginol Node Positive
Number of outliers: 153
     Reginol Node Positive
13                       7
134                      7
166                      7
221                      7
231                      7
--------------------------------------------------
Column: Survival Months
Number of outliers: 0
Empty DataFrame
Columns: [Survival Months]
Index: []
--------------------------------------------

## Capping

In [ ]:
for col in numerical_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap the outliers
    df[col] = df[col].apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the numerical columns
scaler = MinMaxScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


## Feature Selection

In [ ]:
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical features
df_encoded = pd.get_dummies(df.drop('Status', axis=1), drop_first=True)

# Encode the target variable if necessary
df['Status'] = LabelEncoder().fit_transform(df['Status'])

# Calculate Information Gain
info_gain = mutual_info_classif(df_encoded, df['Status'])
print("Information Gain Scores:", info_gain)

# Calculate Chi-Square
chi_scores, p_values = chi2(df_encoded, df['Status'])
print("Chi-Square Scores:", chi_scores)


Information Gain Scores: [0.01286196 0.01012869 0.         0.01099834 0.0851277  0.00534466
 0.         0.00276259 0.00128822 0.         0.         0.00435678
 0.00198096 0.         0.00896255 0.00169567 0.         0.00936423
 0.         0.         0.         0.         0.00071775 0.00144511
 0.00198015 0.         0.00140448 0.010851   0.00402444]
Chi-Square Scores: [6.69112411e-01 2.35709495e+00 9.74367111e-01 7.60549314e+00
 2.62720985e+01 2.16379448e+00 1.71555240e-01 3.37552098e+00
 7.70854272e+00 1.72538423e-03 5.67697326e+00 1.08081742e+01
 3.37363030e-01 8.62150459e+00 1.22941085e+01 6.66677156e+00
 2.26576053e+00 1.11312723e+01 8.62150459e+00 6.66677156e+00
 2.65788055e+01 2.50853036e+00 1.09219313e+01 1.09219313e+01
 3.01864217e+00 2.65788055e+01 4.06656320e-02 3.04802356e+00
 8.81747464e+00]


In [ ]:
import numpy as np

# Combine scores into a DataFrame for easy viewing and selection
feature_scores = pd.DataFrame({
    'Feature': df_encoded.columns,
    'Information_Gain': info_gain,
    'Chi_Square': chi_scores
})

# Sort features by Information Gain and Chi-Square scores
feature_scores = feature_scores.sort_values(by=['Information_Gain', 'Chi_Square'], ascending=False)
top_features = feature_scores.head(10)  # Select top 10 features as an example

print("Top features based on Information Gain and Chi-Square:")
print(top_features)


Top features based on Information Gain and Chi-Square:
                         Feature  Information_Gain  Chi_Square
4                Survival Months          0.085128   26.272098
0                            Age          0.012862    0.669112
3          Reginol Node Positive          0.010998    7.605493
27      Estrogen Status_Positive          0.010851    3.048024
1                     Tumor Size          0.010129    2.357095
17                6th Stage_IIIA          0.009364   11.131272
14                    N Stage_N2          0.008963   12.294108
5                     Race_Other          0.005345    2.163794
11                   T Stage _T2          0.004357   10.808174
28  Progesterone Status_Positive          0.004024    8.817475


In [ ]:
# Assuming you chose the top 10 features based on Information Gain and Chi-Square scores
selected_features = top_features['Feature'].tolist()  # List of top feature names

# Filter the dataset to only include selected features
X = df_encoded[selected_features]
y = df['Status']


In [ ]:
# Step 1: One-hot encode the entire dataset
df_encoded = pd.get_dummies(df, drop_first=True)

# Step 2: Separate features and target after encoding
X = df_encoded.drop('Status', axis=1)
y = df_encoded['Status']

# Step 3: Select the top features from the encoded dataset
top_features_encoded = [feature for feature in X.columns if feature in selected_features]

# Step 4: Use the selected features for training and testing
X = X[top_features_encoded]

# Step 5: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Fit and predict with the KNN model as before
knn = KNN(k=3)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

# Evaluate the accuracy
accuracy = np.mean(predictions == y_test)
print(f"KNN Accuracy: {accuracy * 100:.2f}%")


TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
# Check for any non-numeric columns in X_train or X_test
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
if len(non_numeric_columns) > 0:
    print("Non-numeric columns found:", non_numeric_columns)
else:
    print("All columns are numeric.")


All columns are numeric.


# Knn

In [ ]:
import numpy as np
from collections import Counter

class KNN:
    def __init__(self, k=3):
        self.k = k

    # Fit method simply stores the training data
    def fit(self, X_train, y_train):
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)

    # Predict method for each test sample
    def predict(self, X_test):
        predictions = [self._predict(x) for x in np.array(X_test)]
        return np.array(predictions)

    # Private method to predict the class for a single instance
    def _predict(self, x):
        # Step 1: Compute distances between x and all examples in the training set
        distances = [np.sqrt(np.sum((x - x_train)**2)) for x_train in self.X_train]

        # Step 2: Sort by distance and return indices of the first k neighbors
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Step 3: Majority vote, most common class label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

# Example usage:

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and train the KNN model
knn = KNN(k=3)
knn.fit(X_train, y_train)

# Make predictions on the test set
predictions = knn.predict(X_test)

# Evaluate the accuracy
accuracy = np.mean(predictions == y_test)
print(f"KNN Accuracy: {accuracy * 100:.2f}%")


KNN Accuracy: 91.80%


# Gaussian

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Instantiate the Gaussian Naive Bayes model
nb = GaussianNB()

# Fit the model to the training data
nb.fit(X_train, y_train)

# Make predictions on the test set
nb_predictions = nb.predict(X_test)

# Evaluate the accuracy
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f"Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%")


Naive Bayes Accuracy: 84.14%


# C4.5 Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Instantiate the Decision Tree with entropy criterion
c45_tree = DecisionTreeClassifier(criterion='entropy', random_state=42)

# Fit the model to the training data
c45_tree.fit(X_train, y_train)

# Make predictions on the test set
c45_predictions = c45_tree.predict(X_test)

# Evaluate the accuracy
c45_accuracy = accuracy_score(y_test, c45_predictions)
print(f"C4.5 Decision Tree Accuracy: {c45_accuracy * 100:.2f}%")


C4.5 Decision Tree Accuracy: 84.85%


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate the Random Forest model with 100 trees
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=42)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Make predictions on the test set
rf_predictions = rf.predict(X_test)

# Evaluate the accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")


Random Forest Accuracy: 92.51%


# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Instantiate the Gradient Boosting model
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model to the training data
gb.fit(X_train, y_train)

# Make predictions on the test set
gb_predictions = gb.predict(X_test)

# Evaluate the accuracy
gb_accuracy = accuracy_score(y_test, gb_predictions)
print(f"Gradient Boosting Accuracy: {gb_accuracy * 100:.2f}%")


Gradient Boosting Accuracy: 92.16%


In [ ]:
import pandas as pd

def display_feature_importance(model, model_name):
    # Extract feature importances
    importances = model.feature_importances_
    # Create a DataFrame for feature importance
    feature_importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # Display the top features
    print(f"Top Features by Importance in {model_name}:")
    print(feature_importance_df.head())
    print("\n")

# Random Forest
display_feature_importance(rf_grid_search.best_estimator_, "Random Forest")

# Gradient Boosting
display_feature_importance(gb, "Gradient Boosting")

# C4.5 Decision Tree
display_feature_importance(c45_tree, "C4.5 Decision Tree")


Top Features by Importance in Random Forest:
                        Feature  Importance
3               Survival Months    0.553330
0                           Age    0.159281
1                    Tumor Size    0.124121
2         Reginol Node Positive    0.054234
9  Progesterone Status_Positive    0.030308


Top Features by Importance in Gradient Boosting:
                        Feature  Importance
3               Survival Months    0.710895
0                           Age    0.106226
1                    Tumor Size    0.058949
2         Reginol Node Positive    0.040271
9  Progesterone Status_Positive    0.032136


Top Features by Importance in C4.5 Decision Tree:
                    Feature  Importance
3           Survival Months    0.452935
0                       Age    0.235582
1                Tumor Size    0.139768
2     Reginol Node Positive    0.072460
8  Estrogen Status_Positive    0.027236




# Simple Multi-Layer Perceptron (MLP)

### It can achieve high accuracy on structured/tabular data.

In [ ]:
from sklearn.neural_network import MLPClassifier

# Instantiate the MLP model
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', solver='adam', max_iter=500, random_state=42)

# Fit the model to the training data
mlp.fit(X_train, y_train)

# Make predictions on the test set
mlp_predictions = mlp.predict(X_test)

# Evaluate the accuracy
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
print(f"Neural Network Accuracy: {mlp_accuracy * 100:.2f}%")


Neural Network Accuracy: 92.16%


## Hyperparameter Tuning for Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 150],         # Number of trees
    'max_depth': [None, 10, 20, 30],        # Depth of each tree
    'min_samples_split': [2, 5, 10]         # Minimum samples to split a node
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up Grid Search
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the Grid Search to the data
rf_grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Hyperparameters for Random Forest:", rf_grid_search.best_params_)
print("Best Accuracy for Random Forest:", rf_grid_search.best_score_)


Best Hyperparameters for Random Forest: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best Accuracy for Random Forest: 0.919755209990455


## Hyperparameter Tuning for Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

# Define the hyperparameter grid for Neural Network
nn_param_grid = {
    'hidden_layer_sizes': [(50,), (50, 30), (100, 50)],  # Different layer configurations
    'learning_rate_init': [0.001, 0.01, 0.1],            # Initial learning rate
    'activation': ['relu', 'tanh']                       # Activation function
}

# Initialize the MLPClassifier
nn_model = MLPClassifier(max_iter=500, random_state=42)

# Set up Grid Search
nn_grid_search = GridSearchCV(estimator=nn_model, param_grid=nn_param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the Grid Search to the data
nn_grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Hyperparameters for Neural Network:", nn_grid_search.best_params_)
print("Best Accuracy for Neural Network:", nn_grid_search.best_score_)


Best Hyperparameters for Neural Network: {'activation': 'relu', 'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.1}
Best Accuracy for Neural Network: 0.9126222955774738


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Calculate accuracy for each model
knn_accuracy = accuracy_score(y_test, predictions) * 100
nb_accuracy = accuracy_score(y_test, nb_predictions) * 100
c45_accuracy = accuracy_score(y_test, c45_predictions) * 100
rf_accuracy = rf_grid_search.best_score_ * 100
gb_accuracy = accuracy_score(y_test, gb_predictions) * 100
mlp_accuracy = nn_grid_search.best_score_ * 100

# Define best hyperparameters and feature importance for each model
results_data = {
    "Model": [
        "K-Nearest Neighbors",
        "Naive Bayes",
        "C4.5 Decision Tree",
        "Random Forest",
        "Gradient Boosting",
        "Neural Network"
    ],
    "Accuracy (%)": [
        knn_accuracy,
        nb_accuracy,
        c45_accuracy,
        rf_accuracy,
        gb_accuracy,
        mlp_accuracy
    ],
}

# Create DataFrame
results_df = pd.DataFrame(results_data)

# Display the results
print("Model Performance Summary Table:")
print(results_df)


Model Performance Summary Table:
                 Model  Accuracy (%)
0  K-Nearest Neighbors     91.800357
1          Naive Bayes     84.135472
2   C4.5 Decision Tree     84.848485
3        Random Forest     91.975521
4    Gradient Boosting     92.156863
5       Neural Network     91.262230
